# Main Notebook for running code

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/OptiML/repo/OptML-project

/content/drive/MyDrive/OptiML/repo/OptML-project


## Global Imports

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sklearn
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import datasets, transforms
from sklearn.metrics import accuracy_score, recall_score, f1_score
from Functions.implementations import *

## Load Data

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = SimpleCNN().to(device)
train_loader,valid_loader, test_loader = get_data_loaders()

Train and evaluate a simple model to test if everything works.

## SGD :

In [6]:
import os

#only for google Colab :

# Set a path in your Google Drive
csv_path = "/content/drive/MyDrive/OptiML/repo/OptML-project_SGD_VGGLike_Transform.csv"
save_path = "/content/drive/MyDrive/OptiML/repo/OptML-project/Results/SGD"
# For local machine, set the path to your desired location
#save_path = os.getcwd() + "/Results/SGD"
#csv_path = save_path + "/sgd_gridsearch_results.csv"


epochs = 100
eval_interval = 10

# Create the CSV with headers if it doesn't exist
if not os.path.exists(csv_path):
    columns = ["learning_rate", "momentum"] +  [f"epoch_{i}" for i in range(epochs//eval_interval, epochs + 1, eval_interval)] + ["Test"]
    pd.DataFrame(columns=columns).to_csv(csv_path, index=False)

set_seed(42) # Set a random seed for reproducibility

In [ ]:
from itertools import product


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_loader,valid_loader, test_loader = get_data_loaders(batch_size=128)

results_grid = {}

# learning_rates = [5e-2, 1e-2, 5e-3, 1e-3]
# momentums = [0.0, 0.6, 0.9, 0.99]

learning_rates = [5e-2, 1e-2, 5e-3, 1e-3]
momentums = [0.6]

# Iterate over all combinations of lr and momentum

for lr, momentum in product(learning_rates, momentums):
    scores, model = train_and_return_evaluation_SGD(
        VGGLike,
        lr=lr,
        momentum=momentum,
        train_loader=train_loader,
        valid_loader=valid_loader,
        test_loader=test_loader,
        device=device,
        epochs=epochs,
        eval_interval=eval_interval
    )
    results_grid[(lr, momentum)] = scores

    # Extract F1 scores only, for CSV export
    f1_scores = [f1 for (_, _, _, f1) in scores]

    row = [lr, momentum] + f1_scores

    # Append to CSV
    df_row = pd.DataFrame([row])
    df_row.to_csv(csv_path, mode='a', header=False, index=False)

    # Save the model
    torch.save(model.state_dict(), save_path + f"/VGG_Transform_lr_{lr}_momentum_{momentum}.pth")


🔧 Training with SGD: lr=0.05, momentum=0.6
Epoch 10 | Acc=0.7882 | Recall=0.7883 | F1=0.7868
Epoch 20 | Acc=0.8156 | Recall=0.8157 | F1=0.8124
Epoch 30 | Acc=0.8438 | Recall=0.8440 | F1=0.8440
Epoch 40 | Acc=0.8480 | Recall=0.8485 | F1=0.8488
Epoch 50 | Acc=0.8420 | Recall=0.8423 | F1=0.8423
Epoch 60 | Acc=0.8514 | Recall=0.8507 | F1=0.8505
Epoch 70 | Acc=0.8574 | Recall=0.8575 | F1=0.8574
Epoch 80 | Acc=0.8592 | Recall=0.8589 | F1=0.8592
Epoch 90 | Acc=0.8422 | Recall=0.8424 | F1=0.8438
Epoch 100 | Acc=0.8540 | Recall=0.8544 | F1=0.8545

Test Set Evaluation: Acc=0.8650 | Recall=0.8650 | F1=0.8652

🔧 Training with SGD: lr=0.01, momentum=0.6
Epoch 10 | Acc=0.6124 | Recall=0.6120 | F1=0.6009
Epoch 20 | Acc=0.7536 | Recall=0.7541 | F1=0.7521
Epoch 30 | Acc=0.7876 | Recall=0.7866 | F1=0.7884
Epoch 40 | Acc=0.8230 | Recall=0.8237 | F1=0.8226
Epoch 50 | Acc=0.8238 | Recall=0.8243 | F1=0.8256
Epoch 60 | Acc=0.8378 | Recall=0.8377 | F1=0.8372
Epoch 70 | Acc=0.8350 | Recall=0.8359 | F1=0.8354


# Adam :

In [ ]:
import os

#only for google Colab :

# Set a path in your Google Drive
#csv_path = "/content/drive/MyDrive/OptiML/sgd_gridsearch_results.csv"

# For local machine, set the path to your desired location
save_path = os.getcwd() + "/Results/Adam"
csv_path = save_path + "/adam_gridsearch_results.csv"

# Set number of epochs and evaluation interval
epochs = 100
eval_interval = 10

# Create the CSV with headers if it doesn't exist
if not os.path.exists(csv_path):
    columns = ["learning_rate", "momentum"] +  [f"epoch_{i}" for i in range(epochs//eval_interval, epochs + 1, eval_interval)] + ["Test"]
    pd.DataFrame(columns=columns).to_csv(csv_path, index=False)

set_seed(42) # Set a random seed for reproducibility

In [ ]:
from itertools import product


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_loader, test_loader = get_data_loaders(batch_size=128)

results_grid = {}

# learning_rates = [5e-2, 1e-2, 5e-3, 1e-3]
# betas = [(0.9, 0.999), (0.8, 0.999), (0.9, 0.99)]

learning_rates = [1e-2]
betas = [(0.9, 0.999), (0.8, 0.999), (0.9, 0.99)]

# Iterate over all combinations of lr and momentum
for lr, betas in product(learning_rates, betas):
    scores = train_and_return_evaluation_Adam(
        VGGLike,
        lr=lr,
        betas=betas,
        train_loader=train_loader,
        test_loader=test_loader,
        device=device,
        epochs=epochs,
        eval_interval=eval_interval
    )
    results_grid[(lr, betas[0], betas[1])] = scores

    # Extract F1 scores only, for CSV export
    f1_scores = [f1 for (_, _, _, f1) in scores]

    row = [lr, betas[0], betas[1]] + f1_scores

    # Append to CSV
    df_row = pd.DataFrame([row])
    df_row.to_csv(csv_path, mode='a', header=False, index=False)

    # Save the model
    torch.save(model.state_dict(), save_path + f"/model_lr_{lr}_beta1_{betas[0]}_beta2_{betas[2]}.pth")

Test on colab :

lr = 0.05
- (0.9, 0.999) : F1 ~ 0.18
- (0.8, 0.999) : F1 ~ 0.18
- (0.9, 0.99) : F1 ~

lr = 0.01
- (0.9, 0.999) : F1 ~
- (0.8, 0.999) : F1 ~
- (0.9, 0.99) : F1 ~

lr = 0.005
- (0.9, 0.999) : F1 ~
- (0.8, 0.999) : F1 ~
- (0.9, 0.99) : F1 ~

lr = 0.001
- (0.9, 0.999) : F1 ~
- (0.8, 0.999) : F1 ~
- (0.9, 0.99) : F1 ~